In [8]:
import requests
response = requests.get("https://api.inaturalist.org/v1/observations/species_counts?verifiable=true&spam=false&captive=false&locale=en&preferred_place_id=1&lat=42.947762&lng=-78.782513&radius=50&page=1")
if response.status_code != 200:
    print('Error from API request')

data = response.text

In [32]:
import json
jsonData = json.loads(data)

results = jsonData['results']

In [37]:
import pandas as pd

df = pd.json_normalize(results)

In [41]:
simpleCounts = df[["count","taxon.preferred_common_name"]]
simpleCounts

,count,taxon.preferred_common_name
0,444,Canada Goose
1,434,Monarch
2,428,Eastern Gray Squirrel
3,424,common reed
4,406,Mallard
...,...,...
495,30,hairy white oldfield aster
496,30,Magnolia Warbler
497,30,Black-throated Green Warbler
498,30,Orchard Orbweaver


In [42]:
simpleCounts.to_csv(r'counts.csv',index=False)